<a href="https://colab.research.google.com/github/jonrtaylor/twitch/blob/master/Production_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade numerox
import numerox as nx
import pandas as pd
import numpy as np
from sklearn import linear_model
data = nx.download('numerai_dataset.zip')
X1_train = data['era1':'era30'].x #training data for partition 1
X2_train = data['era31':'era60'].x #training data for partition 2
X3_train = data['era61':'era90'].x #training data for partition 3
X4_train = data['era91': 'era120'].x #training data for partition 4
X_test = data['validation'].x
y1_train = data['era1':'era30'].y['kazutsugi']
y2_train = data['era31':'era60'].y['kazutsugi']
y3_train = data['era61':'era90'].y['kazutsugi']
y4_train = data['era91':'era120'].y['kazutsugi']
y_test = data['validation'].y['kazutsugi']

     |████████████████████████████████| 1.8MB 7.0MB/s 
     |████████████████████████████████| 163kB 47.9MB/s 
  Created wheel for numerox: filename=numerox-4.1.7-cp36-none-any.whl size=1805112 sha256=6d52ff09d47a380a4cf5603ae5e9760b55992c77e98aba2264ba11d4ca1968d3
  Stored in directory: /root/.cache/pip/wheels/b8/d0/3c/1f032762cc54b93e3def50b4f6a49f3f0ad27c3e8d561a4c40
Successfully built numerox


numerai_dataset.zip: 389MB [00:39, 16.3MB/s]                           

In [2]:
L1 = linear_model.LinearRegression()
L2 = linear_model.LinearRegression()
L3 = linear_model.LinearRegression()
L4 = linear_model.LinearRegression()

In [3]:
L1.fit(X1_train, y1_train)
L2.fit(X2_train, y2_train)
L3.fit(X3_train, y3_train)
L4.fit(X4_train, y4_train)
#score function
def correlation_score(y_true, y_pred):
    return np.corrcoef(y_true, y_pred)[0,1]

In [4]:
L1_preds = L1.predict(X_test) #make predictions on the out-of-sample test set
L2_preds = L2.predict(X_test)
L3_preds = L3.predict(X_test)
L4_preds = L4.predict(X_test)

In [5]:
L1_score = correlation_score(y_test, L1_preds) #calculate correlation scores on the out-of-sample predictions
L2_score = correlation_score(y_test, L2_preds)
L3_score = correlation_score(y_test, L3_preds)
L4_score = correlation_score(y_test, L4_preds)

In [6]:
print(L1_score) #each partition's out-of-sample correlation score
print(L2_score)
print(L3_score)
print(L4_score)

0.015131133484415204
0.012834410850005366
0.00992985005947797
0.007843202986115945


In [7]:
preds = pd.DataFrame(L1_preds)
preds.columns = ['L1_preds']
preds['L2_preds'] = L2_preds
preds['L3_preds'] = L3_preds
preds['L4_preds'] = L4_preds
preds['averaged_preds'] = (preds.L1_preds + preds.L2_preds + preds.L3_preds + preds.L4_preds) / 4 #arithmetic average of each model's out of sample predictions

2020-09-08 22:33:14,086 INFO numexpr.utils: NumExpr defaulting to 4 threads.


In [8]:
averaged_vali_score = correlation_score(y_test, preds.averaged_preds) #averaged out-of-sample correlation score; should be higher than any one model's individual score
print(averaged_vali_score)

0.01615743682941112


In [9]:
your_name = "UserName"
tournament_name = "kazutsugi" #don't change
prediction_name = "prediction_kazutsugi" #don't change
preds1 = pd.DataFrame(data['tournament'].ids)
preds1.columns = ['id']
preds1.set_index("id", inplace=True)
preds1['L1_preds'] = L1.predict(data['tournament'].x)
preds1['L2_preds'] = L2.predict(data['tournament'].x)
preds1['L3_preds'] = L3.predict(data['tournament'].x)
preds1['L4_preds'] = L4.predict(data['tournament'].x)
preds1[prediction_name] = (preds1.L1_preds + preds1.L2_preds + preds1.L3_preds + preds1.L4_preds) / 4
preds1[prediction_name].to_csv(tournament_name + "_" + your_name + "_submission.csv", header=True)

In [10]:
#public_id = "REPLACEME"
#secret_key = "REPLACEME"
#napi = numerapi.NumerAPI(public_id=public_id, secret_key=secret_key)
#submission_id = napi.upload_predictions(tournament_name + "_" + your_name + "_submission.csv")

In [11]:
#from google.colab import files
#files.download(tournament_name + "_" + your_name + "_submission.csv")